In [1]:
import numpy as np
import pandas as pd
import os

import git
from git import RemoteProgress

from git import Repo
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
%matplotlib inline

In [2]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [3]:
# uncomment the correct repo you want the information from

# First commits given to us # #

    # # apache poi 
    # remote_link = "https://github.com/apache/poi"
    # local_link = "repo/poi"
    # # Uncomment to clone
    # # Repo.clone_from(remote_link, local_link, progress=Progress())
    # repo = Repo(local_link)
    # fixing_commit = "3a328aa220f6979f9805f658ae33244d153beaa7"
    # affected_files = ["src/ooxml/java/org/apache/poi/POIXMLTypeLoader.java",
    #                   "src/ooxml/java/org/apache/poi/util/DocumentHelper.java"]

    # # cloudfoundry uua 
    # remote_link = "https://github.com/cloudfoundry/uaa"
    # local_link = "repo/uua"
    # # Uncomment to clone
    # # Repo.clone_from(remote_link, local_link, progress=Progress())
    # repo = Repo(local_link)
    # fixing_commit = "14c745aa293b8d3ce9cdd6bfbc6c0ef3f269b21"
    # affected_files = ["server/src/main/java/org/cloudfoundry/identity/uaa/zone/GeneralIdentityZoneConfigurationValidator.java"]

    # # apache struts 
    # remote_link = "https://github.com/apache/struts"
    # local_link = "repo/struts"
    # # Uncomment to clone
    # # Repo.clone_from(remote_link, local_link, progress=Progress())
    # repo = Repo(local_link)
    # fixing_commit = "74e26830d2849a84729b33497f729e0f033dc147"
    # affected_files = ["core/src/main/java/com/opensymphony/xwork2/security/DefaultExcludedPatternsChecker.java",
    #                   "core/src/test/java/com/opensymphony/xwork2/interceptor/ParametersInterceptorTest.java"]

# # new commits given to us # #

# # tom cat
# remote_link = "https://github.com/apache/tomcat"
# local_link = "repo/tomcat"
# # Uncomment to clone
# # Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# fixing_commit = "d2e8f2ede7dea39f75f68384f331f38f094e4ed3"
# affected_files = ["java/org/apache/coyote/ajp/AjpAprProcessor.java", 
#                     "java/org/apache/coyote/ajp/AjpAprProtocol.java",
#                     "java/org/apache/coyote/ajp/AjpProcessor.java",
#                     "java/org/apache/coyote/ajp/AjpProtocol.java",
#                     "java/org/apache/coyote/http11/Http11AprProcessor.java",
#                     "java/org/apache/coyote/http11/Http11AprProtocol.java",
#                     "java/org/apache/coyote/http11/Http11NioProcessor.java",
#                     "java/org/apache/coyote/http11/Http11Processor.java",
#                     "java/org/apache/coyote/http11/Http11Protocol.java"]

# # apache sling (old mirror version)
# remote_link = "https://github.com/apache/sling"
# local_link = "repo/sling-old-svn-mirror"
# # Uncomment to clone
# Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# fixing_commit = "7d2365a248943071a44d8495655186e4f14ea294"
# affected_files = ["bundles/extensions/xss/src/main/java/org/apache/sling/xss/impl/XSSAPIImpl.java"]

# spring-framework
remote_link = "https://github.com/spring-projects/spring-framework"
local_link = "repo/spring-framework"
# Uncomment to clone
Repo.clone_from(remote_link, local_link, progress=Progress())
repo = Repo(local_link)
fixing_commit = "7576274874deeccb6da6b09a8d5bd62e8b5538b7"
affected_files = ["spring-web/src/main/java/org/springframework/http/converter/xml/Jaxb2CollectionHttpMessageConverter.java"]




Receiving objects:   0% (1/544294)
Receiving objects:   0% (1511/544294), 92.01 KiB | 149.00 KiB/s
Receiving objects:   1% (5443/544294), 544.01 KiB | 495.00 KiB/s
Receiving objects:   2% (10886/544294), 1.73 MiB | 1.09 MiB/s
Receiving objects:   2% (12298/544294), 1.73 MiB | 1.09 MiB/s
Receiving objects:   3% (16329/544294), 3.04 MiB | 1.45 MiB/s
Receiving objects:   4% (21772/544294), 4.50 MiB | 1.74 MiB/s
Receiving objects:   4% (24796/544294), 4.50 MiB | 1.74 MiB/s
Receiving objects:   5% (27215/544294), 6.09 MiB | 1.97 MiB/s
Receiving objects:   5% (30782/544294), 7.80 MiB | 2.17 MiB/s
Receiving objects:   6% (32658/544294), 9.63 MiB | 2.35 MiB/s
Receiving objects:   6% (37611/544294), 11.64 MiB | 2.53 MiB/s
Receiving objects:   7% (38101/544294), 11.64 MiB | 2.53 MiB/s
Receiving objects:   8% (43544/544294), 16.03 MiB | 3.44 MiB/s
Receiving objects:   8% (44948/544294), 16.03 MiB | 3.44 MiB/s
Receiving objects:   8% (47057/544294), 20.92 MiB | 3.97 MiB/s
Receiving objects:   9% (

Receiving objects:  95% (517080/544294), 133.07 MiB | 1.65 MiB/s
Receiving objects:  95% (519569/544294), 133.07 MiB | 1.65 MiB/s
Receiving objects:  96% (522523/544294), 133.07 MiB | 1.65 MiB/s
Receiving objects:  97% (527966/544294), 134.54 MiB | 1.75 MiB/s
Receiving objects:  98% (533409/544294), 134.54 MiB | 1.75 MiB/s
Receiving objects:  99% (538852/544294), 134.54 MiB | 1.75 MiB/s
Receiving objects:  99% (542164/544294), 136.17 MiB | 2.04 MiB/s
Receiving objects: 100% (544294/544294), 137.95 MiB | 2.33 MiB/s
Receiving objects: 100% (544294/544294), 138.60 MiB | 3.71 MiB/s, done.
Resolving deltas:   0% (0/273461)
Resolving deltas:   1% (2744/273461)
Resolving deltas:   2% (5560/273461)
Resolving deltas:   3% (8211/273461)
Resolving deltas:   4% (10949/273461)
Resolving deltas:   5% (13687/273461)
Resolving deltas:   6% (16410/273461)
Resolving deltas:   7% (19150/273461)
Resolving deltas:   8% (21906/273461)
Resolving deltas:   9% (24636/273461)
Resolving deltas:  10% (27348/27346

In [4]:
def removeDuplicates(array):
    new_array = []
    for value in array:
        if value not in new_array:
            new_array.append(value)
    return new_array
        

In [5]:
# prints the commit message, 3a

show_data = repo.git.show("-s", fixing_commit).splitlines()
for line in show_data:
    print(line)


commit 7576274874deeccb6da6b09a8d5bd62e8b5538b7
Author: Arjen Poutsma <apoutsma@gopivotal.com>
Date:   Tue Aug 6 15:04:09 2013 -0700

    Fix potential security risk when using Spring OXM
    
    Disable by default external entity resolution when using Spring OXM
    with jaxb. This prevents a XML entity from being able to resolve a
    local file on the host system.
    
    See:
    https://www.owasp.org/index.php/XML_External_Entity_(XXE)_Processing
    
    Issue: SPR-10806


In [6]:
# print the number of files changed, 3b

# retuns the files that have been changed
files = repo.git.whatchanged("-1", "--format=oneline", fixing_commit).splitlines()

# minus one as one is the commit message
files.pop(0)

print(str(len(files)) + " files changed")

1 files changed


In [7]:
# prints the number of unique directories, 3c

all_directories = []

for file in files:
    file_split = file.split()
    full_dir = file_split[5]
    directory = os.path.dirname(full_dir)
    all_directories.append(directory)
    
all_directories = removeDuplicates(all_directories)
print(str(len(all_directories)) + " directories changed")
    

1 directories changed


In [8]:
# prints the number of added/delete lines (inculding comments / blanks lines), 3d, 3e

lines_changed = repo.git.show("--pretty=tformat:", "--numstat", fixing_commit).splitlines()

total_lines_added = 0
total_lines_deleted = 0

for line in lines_changed:
    line_split = line.split()
    total_lines_added += int(line_split[0].strip())
    total_lines_deleted += int(line_split[1].strip())
    
print("Total deletions: " + str(total_lines_deleted))
print("Total additions: " + str(total_lines_added))

Total deletions: 1
Total additions: 3


In [9]:
def countNumberBlankComments(file, langauge):
    total_blank_additions = 0
    total_blank_deletions = 0
    
    if langauge == "Java":
        for line in file:
            if line[0] == "+":
                line = line[1:]
                line = line.strip()
                if line == "":
                    total_blank_additions += 1
                    continue
                if line[0] == "/" or line[0] == "*":
                    total_blank_additions += 1
                    
            if line[0] == "-":
                line = line[1:]
                line = line.strip()
                if line == "":
                    total_blank_deletions += 1
                    continue
                if line[0] == "/" or line[0] == "*":
                    total_blank_deletions += 1
    return total_blank_additions, total_blank_deletions

In [10]:
# prints the number of added/deleted lines (Excluding comments / blanks lines) 3f, 3g

total_blank = [0, 0]

# for each changed file
for file in files:
    # get the directory
    file_split = file.split()
    full_dir = file_split[5]
    
    #get the diff and find the blanks and comments     
    affected_file_diff = repo.git.diff(fixing_commit + "^" + ".." + fixing_commit, "--", full_dir).splitlines()
    this_total_blank = countNumberBlankComments(affected_file_diff, "Java")
    
    # add blanks and comments to total     
    total_blank = np.add(total_blank, this_total_blank)

# get the total from before and the found blank lines and remove them from total
total_blank_additions = total_lines_added - total_blank[0]
total_blank_deletions = total_lines_deleted - total_blank[1]

print("Total deletions (without comments/blanklines): " + str(total_blank_deletions))
print("Total additions (without comments/blanklines): " + str(total_blank_additions))

Total deletions (without comments/blanklines): 1
Total additions (without comments/blanklines): 3


In [11]:
def convertToDate(input_date):
    split = input_date.split()
    year = split[4]
    month = strptime(split[1],'%b').tm_mon
    day = split[2]
    return date(year = int(year), month = month, day = int(day))

In [12]:
# time difference between the fixing commit and the previous commit, 3h

for affected_file in affected_files:
    dates = repo.git.log("--format=%ad", fixing_commit, "--", affected_file).splitlines()

    date1_object = convertToDate(dates[0])
    if len(dates) < 2:
        print("no other changes have been made before")
    else:    
        date2_object = convertToDate(dates[1])
        total_days = abs((date1_object - date2_object).days)
        print("Total days since file", os.path.basename(affected_file), "changed:", total_days)

Total days since file Jaxb2CollectionHttpMessageConverter.java changed: 85


In [13]:
# get the number of times this file has been changed, 3i

for affected_file in affected_files:
    times = repo.git.log("--format=%ad", fixing_commit, "--", affected_file).splitlines()
    print("This file", os.path.basename(affected_file), "has been changed:", len(times), "times")

This file Jaxb2CollectionHttpMessageConverter.java has been changed: 6 times


In [14]:
# what devolpers have modifed this file? 3j
all_files_authors = []
for affected_file in affected_files:
    all_authors = repo.git.log("--format=%an", fixing_commit, "--", affected_file).splitlines()
    authors = removeDuplicates(all_authors)
    
    all_files_authors.extend(authors)
    all_files_authors = removeDuplicates(all_files_authors)
    
    print(os.path.basename(affected_file), "\n\tAuthors:")
    for author in authors:
        print("\t\t", author)
    print("")

Jaxb2CollectionHttpMessageConverter.java 
	Authors:
		 Arjen Poutsma
		 Rob Winch
		 Chris Beams
		 Rossen Stoyanchev



In [15]:
def getAuthorCommits(author_name, df):
    row = df[df["Author"].str.contains(author_name)]
    commits = row["Commit"].iloc[0]
    return int(commits)

In [16]:
# how many has each person commited

commit_authors = repo.git.shortlog("-sne", "--all").splitlines()

author_commits = []

for commit_author in commit_authors:
    commit, author = commit_author.split("\t")
    author_commits.append([int(commit), author])
    
df = pd.DataFrame(author_commits, columns=['Commit', 'Author'])

for author in all_files_authors:
    print(author, ":", getAuthorCommits(author, df), " Commits")



Arjen Poutsma : 1245  Commits
Rob Winch : 38  Commits
Chris Beams : 1111  Commits
Rossen Stoyanchev : 3676  Commits
